# Comparison notebook
This Notebook is designed to compare two methods for probabilistic graphs

In [4]:
import numpy as np
from pathlib import Path
from BAG_Code.BayesianAttackGraph import parse_dot, evidences_to_string
from BAG_Code.Tools_tree import kts_layer_static, kts_layer_original
import BAG_Code.Loopy as Loopy

from pgmpy.inference.ExactInference import BeliefPropagation
from pgmpy.factors.discrete import TabularCPD

The following regroups constants for the runing project.

In [5]:
# Name of the simulation
simulation = "HighLevel"

# Constante for probability one
ONE = 0.999

# Path to the folder containing the tree
path = Path.cwd() / ("Personnal_simulations/output_" + simulation + "/strongly_connected_components/")
basename = "comparison"

# We all read from the file the edges and the nodes
BAG_static, null, nodes_static = parse_dot(open(Path.cwd() / ("Personnal_simulations/output_" + simulation + "_no_HP_rapport/strongly_connected_components/" + "ag-nocycles.dot"), 'r').read(), ONE)
BAG_dynamic, null, nodes_dynamic = parse_dot(open(path /"ag-nocycles.dot", 'r').read(), ONE)

# We add the layers for skills, tools and knowledge
kts_layer_static(BAG_static, ONE, nodes_static)
kts_layer_original(BAG_dynamic, ONE, nodes_dynamic)

The following are the functions defined for this simulation

In [6]:
def deviation(BAG_static, BAG_dynamic, nodes_static, evidences, calculate_probabilities=False):
    for type in evidences.keys():
        evidences[type] = {str(k): 1 for k in evidences[type]}
    # Exact inference
    if calculate_probabilities ==1:
        nodes_wo_evidences = [n for n in nodes_static if n not in evidences['static'].keys()]
        nodes_1 = nodes_wo_evidences[:len(nodes_wo_evidences)//3]
        nodes_2 = nodes_wo_evidences[len(nodes_wo_evidences)//3:(len(nodes_wo_evidences)*2)//3]
        nodes_3 = nodes_wo_evidences[(len(nodes_wo_evidences)*2)//3:]
        prop = {}
        prop['static'] = BeliefPropagation(BAG_static)
        prop['dynamic'] = BeliefPropagation(BAG_dynamic)
        total_prob1 = {}
        total_prob2 = {}
        total_prob3 = {}
        for p in prop.items():
            total_prob1[p[0]] = p[1].query(nodes_1, evidence=evidences[p[0]])
            total_prob2[p[0]] = p[1].query(nodes_2, evidence=evidences[p[0]])
            total_prob3[p[0]] = p[1].query(nodes_3, evidence=evidences[p[0]])

    def probability(node, type):
        prob = 1
        if node in evidences.keys():
            prob = evidences[node]
        else:
            if node in nodes_1:
                prob = total_prob1[type].marginalize([n for n in nodes_1 if n != node], inplace=False).values[1]
            elif node in nodes_2:
                prob = total_prob2[type].marginalize([n for n in nodes_2 if n != node], inplace=False).values[1]
            elif node in nodes_3:
                prob = total_prob3[type].marginalize([n for n in nodes_3 if n != node], inplace=False).values[1]
        return prob
    
    # Display the nodes
    prob_static = []
    prob_dynamic = []
    for node in nodes_static.keys():
            prob_static.append(probability(node, 'static'))
            prob_dynamic.append(probability(node, 'dynamic'))
    return prob_static, prob_dynamic

In [7]:
def variation(prob_static, prob_dynamic):
    sum = 0
    lenght = 0
    for i in range(len(prob_static)):
        if prob_static[i] <0.9:
            dif = 2 * abs(prob_dynamic[i] - prob_static[i])/(prob_static[i] + prob_dynamic[i])
            sum += dif
            lenght += 1
    return sum/lenght

def difference(prob_static, prob_dynamic):
    sum = 0
    lenght = 0
    for i in range(len(prob_static)):
        if prob_static[i] <0.9:
            dif = abs(prob_dynamic[i] - prob_static[i])
            sum += 100*dif
            lenght += 1
    return sum/lenght
    

In [8]:
evidences_static = [[11],[11, 9], [11, 9,  7, 5]]
evidences_dynamic = [[11],[11, 9], [11, 9, 7, 19]]
for i in range(1, 4):
    prob_static, prob_dynamic = deviation(BAG_static, BAG_dynamic, nodes_static, {'static' : evidences_static[i-1], 'dynamic' : evidences_dynamic[i-1]}, calculate_probabilities=1)
    var = variation(prob_static, prob_dynamic)
    dif = difference(prob_static, prob_dynamic)
    # We print directly in a markdown format
    print("     \\hline Step ", i, " & ", "{:.1f}".format(dif), "\\% & ", int(var * 100), "\\% \\\\")


     \hline Step  1  &  1.1 \% &  113 \% \\
     \hline Step  2  &  27.2 \% &  142 \% \\
     \hline Step  3  &  60.9 \% &  178 \% \\
